In [99]:
import pandas as pd
import numpy as np
from pandas import DataFrame

from googletrans import Translator 
pd.set_option('display.html.table_schema', True)

# 전처리 가설
## shop.csv 파일을 살펴본 결과 단순한 스토어 개념이 아닌, 각 레벨 별 계층적 구조로 있는것으로 확인, 중간에는 온라인스토어또한 있어서 그에 대한 고려가 필요할듯. 

### 특히 한 컬럼으로만 되어있는 store name이 실제로는 Region을 반영한 이름이었음. 구글 번역기를 통해 영어로 번역한 후  각 store를 카테고리화홤.

In [100]:
shop_df = pd.read_csv('../data/shops_trans.csv', encoding='utf-8')
shop_df = shop_df.drop(["shop_name"], axis=1)

In [101]:
shop_df.head()

,shop_id,en_shop
0,0,"Yakutsk Ordzhonikidze, 56 francs"
1,1,"Yakutsk TC ""Central"" franc"
2,2,"Adygea Shopping Center ""Mega"""
3,3,"Balashikha TRK ""October-Kinomir"""
4,4,"Volzhsky shopping center ""Volga Mall"""


In [102]:
def split(name):
    sptname = name.split(' "')
    lastspt = sptname[0].split(" ")
    return lastspt[0]
#     if '"' in name:
#         return ame.('"')
#     else:
#         return name
shop_df["Region"] = list(map(split, shop_df.en_shop))

## 몇가지 궁금한 사항 : 아래를 보면 10번과 11번은 거의 동일해보인다. 과연 이 두 데이터는 중복되었는가? 아니면 각자의 의미가 있는것인가?

In [103]:
shop_df.iloc[8:13]

,shop_id,en_shop,Region
8,8,"Voronezh TRC City-Park ""Grad""",Voronezh
9,9,Outbound Trade,Outbound
10,10,Zhukovsky Str. Chkalov 39m?,Zhukovsky
11,11,Zhukovsky Str. Chkalov 39 m²,Zhukovsky
12,12,Online shop Emergency,Online


In [104]:
sales_df = pd.read_csv("../data/sales_train_v2.csv")

In [105]:
merged_sample = pd.merge(sales_df, shop_df, on = "shop_id")
merged_sample.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,en_shop,Region
0,02.01.2013,0,59,22154,999.0,1.0,"Yaroslavl Shopping center ""Altair""",Yaroslavl
1,10.01.2013,0,59,22151,399.0,1.0,"Yaroslavl Shopping center ""Altair""",Yaroslavl
2,04.01.2013,0,59,5603,699.0,1.0,"Yaroslavl Shopping center ""Altair""",Yaroslavl
3,19.01.2013,0,59,5587,199.0,2.0,"Yaroslavl Shopping center ""Altair""",Yaroslavl
4,31.01.2013,0,59,5613,5571.0,1.0,"Yaroslavl Shopping center ""Altair""",Yaroslavl


In [106]:
date_df = pd.DataFrame(merged_sample["date"].str.split(".").tolist(),  columns = ['day','month','year'])
date_df.head()

,day,month,year
0,02,01,2013
1,10,01,2013
2,04,01,2013
3,19,01,2013
4,31,01,2013


In [107]:
pre_data = pd.merge(merged_sample, date_df, left_index=True, right_index=True)
pre_data.head()
del pre_data["date_block_num"]
del pre_data["date"]
pre_data.head()

,shop_id,item_id,item_price,item_cnt_day,en_shop,Region,day,month,year
0,59,22154,999.0,1.0,"Yaroslavl Shopping center ""Altair""",Yaroslavl,02,01,2013
1,59,22151,399.0,1.0,"Yaroslavl Shopping center ""Altair""",Yaroslavl,10,01,2013
2,59,5603,699.0,1.0,"Yaroslavl Shopping center ""Altair""",Yaroslavl,04,01,2013
3,59,5587,199.0,2.0,"Yaroslavl Shopping center ""Altair""",Yaroslavl,19,01,2013
4,59,5613,5571.0,1.0,"Yaroslavl Shopping center ""Altair""",Yaroslavl,31,01,2013


In [108]:
AD = pd.DataFrame(pre_data.groupby(["Region", "en_shop", "year", "month"])["item_cnt_day"].sum())
AD.head()

item_cnt_day
Region en_shop                       year month              
Adygea Adygea Shopping Center "Mega" 2013 01           1146.0
                                          02            488.0
                                          03            753.0
                                          04            583.0
                                          05            553.0

## moscow만 봐도 ("Moscow Shopping Center "Silver House"   / Moscow TC "Budenovskiy" (pav.A2) 연속되지 않은 데이터가 있다. 이말 인 즉슨 카테고리 코드로서는 남아있지만 폐점되었을 가능성이 있는 가게들이 있다는 것이다.

In [109]:
temp_moscow = AD[AD.index.get_level_values(0).values == "Moscow"]
pd.set_option('display.max_rows', 500)
temp_moscow

item_cnt_day
Region en_shop                                year month              
Moscow Moscow "Sale"                          2014 10           3261.0
                                              2015 10           2611.0
       Moscow MTRTS "Afi Mall"                2013 01           1495.0
                                                   02           1854.0
                                                   03           2233.0
                                                   04           1846.0
                                                   05           1646.0
                                                   06           1912.0
                                                   07           1860.0
                                                   08           2061.0
                                                   09           2486.0
                                                   10           2015.0
                                                   11           2174.0
                                                   12           3623.0
                                              2014 01           1844.0
                                                   02           1822.0
                                                   03           2088.0
                                                   04           2144.0
                                                   05           1909.0
                                                   06           1849.0
                                                   07           1603.0
                                                   08           1749.0
                                                   09           1897.0
                                                   10           2031.0
                                                   11           2222.0
                                                   12           3959.0
                                              2015 01           1923.0
                                                   02           1839.0
                                                   03           1687.0
                                                   04           2053.0
                                                   05           1717.0
                                                   06           1844.0
                                                   07           1741.0
                                                   08           1622.0
                                                   09           1900.0
                                                   10           1912.0
       Moscow Shop С21                        2013 01           1924.0
                                                   02           2447.0
                                                   03           2588.0
                                                   04           2297.0
                                                   05           1755.0
                                                   06           2243.0
                                                   07           1903.0
                                                   08           2017.0
                                                   09           2518.0
                                                   10           2364.0
                                                   11           2224.0
                                                   12           2618.0
                                              2014 01           1522.0
                                                   02           2007.0
                                                   03           1982.0
                                                   04           1826.0
                                                   05           1496.0
                                                   06           1246.0
                                                   07           1243.0
           

## 또한 Zhukovsky의 경우 코드가 두개로 붙어있는데, 한 코드의 경우 2015년 2월에 임시적으로 사용된 코드로서, 2015년 11월과 12울에는 사용되지 않을 가능성이 매우 높다.

In [110]:
AD[AD.index.get_level_values(0).values == "Zhukovsky"].head(10)

item_cnt_day
Region    en_shop                      year month              
Zhukovsky Zhukovsky Str. Chkalov 39 m² 2015 02            572.0
          Zhukovsky Str. Chkalov 39m?  2013 01            935.0
                                            02           1026.0
                                            03           1017.0
                                            04            756.0
                                            05            683.0
                                            06            764.0
                                            07            800.0
                                            08            755.0
                                            09           1027.0

In [111]:
#pre_data에서 Train Code에서 year에서 2015년이 없는 SHOP을 찾고싶다! 
find2015 = AD.reset_index()
only2015 = find2015.groupby(["year", "en_shop"]).count()
only2015 = only2015[only2015.index.get_level_values(0) == '2015']

In [112]:
problem2015 = only2015[only2015["month"] < 10]
problem2015.sort_values(by=["month"])

Region  month  item_cnt_day
year en_shop                                                                  
2015 Krasnoyarsk Shopping center "Vzletka Plaza"        1      1             1
     Moscow "Sale"                                      1      1             1
     Novosibirsk TRC "Gallery Novosibirsk"              1      1             1
     RostovNaDonu TRC "Megacenter Horizon" Island       1      1             1
     SPb TK "Sennaya"                                   1      1             1
     Zhukovsky Str. Chkalov 39 m²                       1      1             1
     Moscow TP TPP "Perlovski"                          2      2             2
     Outbound Trade                                     2      2             2
     Khimki ТЦ "Mega"                                   4      4             4
     Mytischi TRK "XL-3"                                4      4             4
     Moscow TC "New Century" (Novokosino)               5      5             5
     Moscow ТЦ "MEGA Belaya Dacha II"                   8      8             8
     Tyumen Shopping Center "Green Beach"               9      9             9
     Zhukovsky Str. Chkalov 39m?                        9      9             9

In [113]:
AD[AD.index.get_level_values(1).values == 'Krasnoyarsk Shopping center "Vzletka Plaza"'].tail() #1월에 폐업한듯
AD[AD.index.get_level_values(1).values == 'Moscow "Sale"'] #매년 10월에만 있음
AD[AD.index.get_level_values(1).values == 'Novosibirsk TRC "Gallery Novosibirsk"'] #2015년 10월에 첫 개장
AD[AD.index.get_level_values(1).values == 'Novosibirsk TRC "Gallery Novosibirsk"'] #2015년 10월에 첫 개장

,,,,item_cnt_day
Region,en_shop,year,month,
Novosibirsk,"Novosibirsk TRC ""Gallery Novosibirsk""",2015,10,330.0


## 총 60개가 있던 Shoplist들 중에, 2015년이 있는것들은 54개, 
## 그중에서 10개가 안되는 것은 14개나 된다. 
## 대체 무슨일이있던거야...

In [114]:
shop_df.shape, only2015.shape, problem2015.shape

((60, 3), (54, 3), (14, 3))

In [115]:
problemlist = problem2015.index.get_level_values(1).tolist()
for store in problemlist:
    print(AD[AD.index.get_level_values(1).values == store].tail())
#여기서 살릴 가게들을 결정해야할것 같아요. 몇몇개는 폐업한듯하고 몇몇개는 아닌거같고...

                                    item_cnt_day
Region en_shop          year month              
Khimki Khimki ТЦ "Mega" 2014 12           7677.0
                        2015 01           6043.0
                             02           4221.0
                             03           4625.0
                             04            732.0
                                                                    item_cnt_day
Region      en_shop                                     year month              
Krasnoyarsk Krasnoyarsk Shopping center "Vzletka Plaza" 2014 09           1218.0
                                                             10            997.0
                                                             11           1199.0
                                                             12           1832.0
                                                        2015 01            689.0
                                 item_cnt_day
Region en_shop       year month             

## 아이디어:
    어떻게보면 현재 폐업한 샵도 있지만, Rigon별로 구분했을때는 큰 챠이가 있을까? Region별로 구분했을때는 구매의 총량이 비슷하지 않을까?  각 스토어별로 구매량을 구하는것이긴 하지만 Region에서 가감하는 방법? 어떻게설명해얗라지 모르겠지만 그런 방법도 의미가 있지 않을까...

In [116]:
df_category = pd.read_csv("../data/item_categories_trans.csv")
df_category
del df_category["item_category_name"]
df_category["en_category"].replace(to_replace='Книги - Путеводители', value='Books - Guide', inplace=True)
df_category["en_category"].replace(to_replace='Payment Cards - Windows (Digital)', value='Payment cards - Windows (Digital)', inplace=True)
df_category["en_category"].replace(to_replace='Payment cards (Movies, Music, Games)', value='Payment cards - (Movies, Music, Games)', inplace=True)
df_category

,item_category_id,en_category
0,0,PC - Headsets / Headphones
1,1,Accessories - PS2
2,2,Accessories - PS3
3,3,Accessories - PS4
4,4,Accessories - PSP
5,5,Accessories - PSVita
6,6,Accessories - XBOX 360
7,7,Accessories - XBOX ONE
8,8,Tickets (figure)
9,9,Delivery of goods


In [117]:
#큰카테고리별로 나누기 당장은 큰 카테고리별로 나누기 
def splitcategory(x):
    if '-' in x:
        a = x.split('-')
        for i in a:
            i.strip()
        return a[0]
    else:
        return x

질문 : 두개의 변수를 map에서 받을려면 어떻게 해야하나요..? 예를들어 big_cate랑 small cate해서 a[0]를 bigcate라는 칼럼에, a[1]을 smallcate라는 칼럼에 넣으려면?

In [124]:
df_category["big_cate"] = list(map(splitcategory, df_category["en_category"]))
a = pd.DataFrame(df_category.groupby(["big_cate", "en_category"])["item_category_id"].first())
bigcate = a.index.get_level_values(0).unique().tolist()

In [147]:
df_bigcate = pd.DataFrame(bigcate, index=range(len(bigcate)))
df_bigcate.rename(columns={0 : "big_cate"}, inplace=True)
df_bigcate["idx_ofbigcate"] = df_bigcate.index
df_category

,item_category_id,en_category,big_cate
0,0,PC - Headsets / Headphones,PC
1,1,Accessories - PS2,Accessories
2,2,Accessories - PS3,Accessories
3,3,Accessories - PS4,Accessories
4,4,Accessories - PSP,Accessories
5,5,Accessories - PSVita,Accessories
6,6,Accessories - XBOX 360,Accessories
7,7,Accessories - XBOX ONE,Accessories
8,8,Tickets (figure),Tickets (figure)
9,9,Delivery of goods,Delivery of goods


In [150]:
merged_category = pd.merge[df_category, df_bigcate, on ='big_cate', how='left']

# 흑..왜 안되죠...

SyntaxError: invalid syntax (<ipython-input-150-bb8fcac68edc>, line 1)

## 1. 카테고리를 큰분류별로 나눠서, 팔리는 수치를 확인 (상관없이, 전체회사를 보는느낌으로)

## 2.큰카테고리 / 작은카테고리  월별로 제일 많이 팔리는 것들?

## 3.Region 별로 큰카테고리가 차이가 있는지?

## 4.폐업하고 난 다음에 Region의 판매 총량이 많이 변했는지? 변헀다면 Online으로의 파급효과는 어떘는지?